## Feature Selection
```In this exercise you will experience a very under rated aspect of the data scientist job: feature selection.
We will look at the most common algorithms, examine them and will develop new feature selection algorithms.
The data you will be working on is pretty hard: you will soon find out why :)```

```Note: When questions are asked (you can identify a question by '?'), answer it in another cell and be prepared to talk about it with your tutor.```

```~Ittai Haran```

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

```Load data.csv. The data is made up from some measurements of soil in Africa, and was derived from Kaggle. SOC is the target.
```

In [ ]:
df = pd.read_csv('resources/data.csv')

```Train a simple Random Forest model on the data. Use your knowledge to make it as good a model as you can. Don't forget to split the data to train and test segments.```

```Use model.feature_importances_ to select the 100 most important features. Use these features to train your model. Did you get better results on the train segment? what are results on the test segment?
How does model.feature_importances_ work? Some features have 0 importance. Why?```

```Well, that didn't work quite so good. Let's try a different approach. Now select the 100 features that are most correlated to the data. What results did you get now? You got so far two groups of feature selected. Are they similar to one another? Why or why not?```

```Now cluster the features by their correlation to one another. Make sure you have at least ~30 clusters. Pick from each cluster the feature most correlated to the target. What are your results now? What clustering algorithm did you use?```

```Before we continue, read the documentation of sklearn.feature_selection. What other feature selection algorithms did you find there? try one of them on your data.```

```You are about to be very surprised. select 100 features randomly and evaluate the model trained with them. Repeat it 100 times. What is the best set of feature you got? How is it compared to the other subsets you got so far? Can you explain it?```

```Now it's your time to enter the office's competition for feature selection. Invent your own feature selection algorithm. Make it a good one (because if it isn't, your tutor might ask you to implement one of the other algorithms proposed by the office's members, and that might be awful ;) ) Before you act, talk about your idea with your tutor.```